In [1]:
import math
import csv
import folium

from geopy.distance import geodesic

In [2]:
def get_hexagon_row_exp(start_coord, radius, count):
    coord_data = []
    for _ in range(count):
        next_coord = geodesic(meters=radius * 2).destination(start_coord, 90)
        coord_data.append([next_coord.latitude, next_coord.longitude])
        start_coord = next_coord
    return coord_data

def row_mapper_exp(start_coord, radius, row_count, height_count):
    coord_data = []
    height = radius * math.sqrt(3)
    for _ in range(height_count):
        direction = 90 if _ % 2 != 0 else 270
        next_coord = geodesic(meters=radius).destination(
            geodesic(meters=height).destination(start_coord, 180), direction
        )
        coord_data.extend(get_hexagon_row_exp(next_coord, radius, row_count))
        start_coord = next_coord
    return coord_data

def data_to_csv(data):
    file_path = 'coordinates/hexagonal-packing-v3.csv'
    with open(file_path, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['lat', 'lon'])
        writer.writerows(data)
        print(f"Coordinates saved to {file_path}")

In [3]:
r = 500
start_point = (-8.096568, 114.437477)
row_count = 140
height_count = 100
coordinates = row_mapper_exp(start_point, r, row_count, height_count)

In [4]:
print(len(coordinates))

14000


In [5]:
# TEST BENCH

import os
import csv
import math
import folium
import pandas as pd
import geopandas as gpd

from shapely.geometry import Point
from geopy.distance import geodesic


def get_hexagon_row(start_coord, radius, count):
    coord_data = []
    for _ in range(count):
        next_coord = geodesic(meters=radius * 2).destination(start_coord, 90)
        coord_data.append([next_coord.latitude, next_coord.longitude])
        start_coord = next_coord
    return coord_data


def create_hexagonal_grid(start_coord, radius, row_count, height_count):
    coord_data = []
    height = radius * math.sqrt(3)
    for _ in range(height_count):
        direction = 90 if _ % 2 == 0 else 270
        next_coord = geodesic(meters=radius).destination(
            geodesic(meters=height).destination(start_coord, 180), direction
        )
        coord_data.extend(get_hexagon_row(next_coord, radius, row_count))
        start_coord = next_coord
    return coord_data


def save_to_csv(data, file_path):
    with open(file_path, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['latitude', 'longitude'])
        writer.writerows(data)


def filter_points(land_gdf, forest_gdf, points_df):
    land_points_df = gpd.GeoDataFrame(points_df, geometry=gpd.points_from_xy(points_df.longitude, points_df.latitude))
    land_points_df.set_crs(crs=land_gdf.crs, inplace=True)
    points_in_land = gpd.sjoin(land_points_df, land_gdf, how='inner', op='within')
    points_in_land_index = points_in_land.index
    points_out_forest = points_df[~points_df.index.isin(points_in_land_index)]
    return points_out_forest


def main():
    circle_radius = 500
    starting_point = (-8.096568, 114.437477)
    square_length = 140
    square_height = 100

    square_coordinates = create_hexagonal_grid(starting_point, circle_radius, square_length, square_height)
    coord_csv_path = 'coordinates/bali_island_square.csv'
    save_to_csv(square_coordinates, coord_csv_path)

    forest_dir = 'shape-filter'
    shapefiles = [os.path.join(forest_dir, file) for file in os.listdir(forest_dir) if file.endswith('.shp')]

    square_df = pd.read_csv(coord_csv_path)

    land_gdf = gpd.read_file('land-filter/indonesia_province.shp')

    forest_gdf = gpd.GeoDataFrame()
    for file in shapefiles:
        forest_gdf = pd.concat([forest_gdf, gpd.read_file(file)], ignore_index=True)

    filtered_forest = filter_points(land_gdf, forest_gdf, square_df)

    filtered_map = folium.Map(location=[filtered_forest['latitude'].iloc[0], filtered_forest['longitude'].iloc[0]], zoom_start=10)
    for lat, lon in zip(filtered_forest['latitude'], filtered_forest['longitude']):
        folium.Marker([lat, lon]).add_to(filtered_map)

    filtered_map.save('map-export/bali_island_filtered_experimental.html')


if __name__ == "__main__":
    main()


C:\Users\angger\AppData\Local\Temp\ipykernel_13256\541454588.py:83: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  main()


In [ ]:
test = pd.read_csv()